##Proyecto Individual Nro 1
+ Data 07- Giselle Pereira Nuñez

Sistema de recomendación:
- limpieza
- preparación

In [1]:
#Librerias que voy a necesitar
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [2]:
#Abro el csv con el que voy a trabajar
Proveedores_Resenias= pd.read_csv ("archivo.csv")

C:\Users\Jhonatan\AppData\Local\Temp\ipykernel_11584\2681694485.py:2: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  Proveedores_Resenias= pd.read_csv ("archivo.csv")


In [3]:
# normalizo columna score
scaler = MinMaxScaler(feature_range=(1, 5))
Proveedores_Resenias['score_y'] = scaler.fit_transform(Proveedores_Resenias[['score']])
Proveedores_Resenias['score_y'] = Proveedores_Resenias['score_y'].round(1)

In [6]:
Proveedores_Resenias.head(5)

,id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type,userId,score,timestamp,movieId,plataform,score_y
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,543,5.0,2003-07-30,as1,amazon,5.0
1,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,595,3.0,1996-08-13,as1,amazon,3.2
2,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,611,3.0,2001-01-03,as1,amazon,3.2
3,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,2523,3.5,2012-06-25,as1,amazon,3.7
4,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,3082,4.0,2000-03-30,as1,amazon,4.1


**¿Todos los usiarios evaluaron las peliculas que vieron?**

In [8]:
Proveedores_Resenias['score'].isna().sum()
#si, todos los usuarios calificaron las peliculas

0

**¿hay registros duplicados?**

In [9]:
duplis = Proveedores_Resenias.duplicated(subset=['userId', 'id']).sum()
print(f"Hay {duplis} registros duplicados")
# Si, Hay 129089 registros duplicados, los cuales voy a eliminar para evitar un sesgo en el modelo

Hay 129089 registros duplicados


In [4]:
Proveedores_Resenias = Proveedores_Resenias.drop_duplicates(subset=['userId', 'id'])

**Codificación de datos categóricos, en duration_type**

In [5]:
# Crear un objeto OneHotEncoder y ajustarlo a los valores de la columna 'duracion'
enc = OneHotEncoder()
enc.fit(Proveedores_Resenias[['duration_type']])

OneHotEncoder()

In [12]:
#Valores unicos
Proveedores_Resenias['duration_type'].unique()

array(['min', 'season', nan], dtype=object)

In [6]:
#se encuentran valeres nulos en la columa de duration_type y al traer sus filas tiene tambien muchos otros valores nulos en las otras columnas asi que opto por eliminarlas
masc= Proveedores_Resenias['duration_type'].isna()

In [7]:
Proveedores_Resenias[masc]

#codificar duration_type
def cod (tipo):
    if tipo == "min":
        return 1
    else:
        return 2

In [8]:
Proveedores_Resenias['duration_type_cod'] = Proveedores_Resenias['duration_type'].apply(cod)

In [16]:
Proveedores_Resenias.head(1)

,id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type,userId,score,timestamp,movieId,plataform,score_y,duration_type_cod
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,543,5.0,2003-07-30,as1,amazon,5.0,1


**Codificación de datos categóricos, en type**

In [17]:
#Valores unicos
Proveedores_Resenias['type'].unique()

array(['movie', 'tv show'], dtype=object)

In [9]:
Proveedores_Resenias = pd.get_dummies(Proveedores_Resenias, columns=['type'])

**Normalización de datos en duración de las películas y series**

In [10]:
# columna a normalizar
duracion = Proveedores_Resenias['duration_int']

In [11]:
#instancia del normalizador
normalizador = MinMaxScaler()

In [12]:
#  normalización
duracion_normalizada = normalizador.fit_transform(duracion.values.reshape(-1, 1))

In [13]:
# Crea una nueva columna con los valores normalizados
Proveedores_Resenias['duration_int_norm'] = duracion_normalizada

In [23]:
Proveedores_Resenias.head(1)

,id,title,director,cast,country,date_added,release_year,rating,listed_in,description,...,userId,score,timestamp,movieId,plataform,score_y,duration_type_cod,type_movie,type_tv show,duration_int_norm
0,as1,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,...,543,5.0,2003-07-30,as1,amazon,5.0,1,1,0,0.18802


**Selección de atributos, eliminacion de campos que no contribullen en el modelo de ML**

In [14]:
#elimino los campos que no voy a necesitar
Proveedores_Resenias= Proveedores_Resenias.drop(['director', 'cast', 'date_added', 'description', 'timestamp', 'movieId'], axis=1)

In [15]:
#sigo eliminando otras mas
Proveedores_Resenias= Proveedores_Resenias.drop(['country', 'listed_in'], axis=1)

In [16]:
#categorizo plataform
#Codificación one-hot
Proveedores_Resenias = pd.get_dummies(Proveedores_Resenias, columns=['plataform'])

In [17]:
#reviso los datos unicos de la columna rating
Proveedores_Resenias['rating'].unique()

array(['g', '13+', 'all', '18+', 'r', 'tv-y', 'tv-y7', 'nr', '16+',
       'tv-pg', '7+', 'tv-14', 'tv-nr', 'tv-g', 'pg-13', 'tv-ma', 'pg',
       'nc-17', 'unrated', '16', 'ages_16_', 'ages_18_', 'all_ages',
       'not_rate', 'tv-y7-fv', 'not rated', '2 seasons', '93 min',
       '4 seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 seasons', '121 min', '88 min', '101 min',
       '1 season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 min', '28 mi

In [ ]:
#este tiene datos mal cargados, asi que mi solucion es eliminarla ya qye los datos mal cargados son demasiados
Proveedores_Resenias = Proveedores_Resenias.drop('rating')

In [18]:
#elimino la columna duration_int ya que se encuentra normalizada en la columna duration_int_norm
Proveedores_Resenias = Proveedores_Resenias.drop('duration_int', axis=1)

In [19]:
#elimino la columa de duration_type_cod por que hare otro tipo de categorizacion
Proveedores_Resenias = Proveedores_Resenias.drop('duration_type_cod', axis=1)

In [20]:
# Codificación one-hot para la columna de duration_type
Proveedores_Resenias = pd.get_dummies(Proveedores_Resenias, columns=['duration_type'])

In [22]:
Proveedores_Resenias.head()

,id,title,release_year,rating,userId,score,score_y,type_movie,type_tv show,duration_int_norm,plataform_amazon,plataform_disney_plus,plataform_hulu,plataform_netflix,duration_type_min,duration_type_season,Decada
0,as1,the grand seduction,2014,g,543,5.0,5.0,1,0,0.18802,1,0,0,0,1,0,Década de los 2010
1,as1,the grand seduction,2014,g,595,3.0,3.2,1,0,0.18802,1,0,0,0,1,0,Década de los 2010
2,as1,the grand seduction,2014,g,611,3.0,3.2,1,0,0.18802,1,0,0,0,1,0,Década de los 2010
3,as1,the grand seduction,2014,g,2523,3.5,3.7,1,0,0.18802,1,0,0,0,1,0,Década de los 2010
4,as1,the grand seduction,2014,g,3082,4.0,4.1,1,0,0.18802,1,0,0,0,1,0,Década de los 2010


In [23]:
#elimino la columna release_year porque ya esta categorizada
df_ml = Proveedores_Resenias.drop('release_year', axis=1)

Modelo de ML
- Voy a usar el filtrado colaborativo, una técnica de recomendación que se basa en el comportamiento y las preferencias de los usuarios para recomendar 
elementos similares a los que ya han sido evaluados positivamente por otros usuarios con gustos similares.

In [24]:
#extraigo las columnas que voy a necesitar
df_ml_nuev = df_ml.loc[:, ['userId', 'score_y', 'title', 'id']]

In [25]:
df_ml_nuev.head(1)

,userId,score_y,title,id
0,543,5.0,the grand seduction,as1


In [27]:
# Cargar los datos en el formato esperado por Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ml_nuev[['userId', 'id', 'score_y']], reader)

In [28]:
# Entrenar el modelo SVD, con esos parametros ya que al ser un df muy grande este tarda mucho
model = SVD(n_factors=10, n_epochs=3, lr_all=0.002)
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# Predecir la calificación que un usuario daría a una película
userId = 1
movie_title = 'the grand seduction'
movie_id = df_ml_nuev.loc[df_ml_nuev['title'] == movie_title, 'id'].iloc[0]
predicted_score = model.predict(userId, movie_id).est

In [ ]:
predicted_score
#la Prediccion le dice que tiene un 3.648789214654395 por ciento que le guste la pelicula

In [ ]:
#elimino los duplicados ml.parquet es demasiado grande
df_ml_nuev['title'].duplicated().sum()
#el nro de duplicados es 10873158

In [ ]:
df_ml_nuev.drop_duplicates(subset=['title'], keep='first', inplace=True)

In [ ]:
df_ml_nuev.shape

In [ ]:
#Guardar archivo para predecir en un archivo tipo parquet
df_ml_nuev.to_parquet('ml.parquet')